__Introduction__

Time in force (TIF) is a special instruction to indicate how long an order will remain live before it is executed, cancelled or expires. As a result, it gives the trader or the electronic algorithm a mechanism of controlling time for an order. A few selected TIF values include:

* Day
* Good Till Cancel (GTC)
* At the Opening (OPG)
* Immediate Or Cancel (IOC)
* At the Close (CLS)

The FIX tag for time in force is 59 and additional details are available here.
***

__Question__

In algorithmic trading, a Smart Order Router (SOR) creates child orders and sends them to different exchanges for execution and each child order has an attribute called tif. The function simOrdersTIF simulates a list of child orders with two attributes: **time for order creation time** and **tif for the order's time in force**.

In [1]:
simOrdersTIF:{
    nOrders:10000;
    seed:-314159;
    openTime:`time$09:30;
    closeTime:`time$16:00;
    listTifs:`Day,20#`IOC;

    system "S ",string seed;
    times:asc closeTime&openTime+nOrders?390*60*1000;

    system "S ",string seed;
    tifs:nOrders?listTifs;
    ([] time:times;tif:tifs)
  };
orders:simOrdersTIF[];

In [2]:
orders

time         tif
----------------
09:30:02.550 IOC
09:30:05.444 IOC
09:30:07.042 IOC
09:30:17.080 IOC
09:30:17.418 IOC
09:30:20.919 IOC
09:30:21.349 IOC
09:30:22.178 IOC
09:30:23.478 IOC
09:30:23.567 IOC
09:30:26.683 IOC
09:30:29.890 IOC
09:30:30.155 IOC
09:30:30.308 IOC
09:30:30.320 IOC
09:30:31.384 IOC
09:30:32.263 Day
09:30:37.249 IOC
09:30:38.124 IOC
09:30:38.184 IOC
..


**Question**

Find all __clusters of IOC orders__ and the __cluster with the longest duration__. <br/>
A **cluster of IOC orders is defined as a group of IOC orders created in a row**. A **cluster's duration is the time difference** between the first order and the last order within a cluster.
***

1. Line 1 updates the table with a boolean flag to indicate whether an order is an IOC order.
2. Line 2 creates an IOC group for each IOC cluster. The snippet 1_(>)prior (0,isIOC) flags the first IOC order in an IOC cluster. The sums simply creates an increasing group index for each IOC cluster.
3. Line 3 finds the time duration of each IOC clusters.
4. Line 4 gives the IOC clusters with maximal duration.

In [3]:
orders:update iocLoc:`IOC=tif from orders;
iocCluster:select time, tif, iocLoc from orders

In [7]:
iocClusters:update iocGroup:sums 1_(>)prior (0,iocLoc) from iocCluster;
iocClusters

time         tif iocLoc iocGroup
--------------------------------
09:30:02.550 IOC 1      1       
09:30:05.444 IOC 1      1       
09:30:07.042 IOC 1      1       
09:30:17.080 IOC 1      1       
09:30:17.418 IOC 1      1       
09:30:20.919 IOC 1      1       
09:30:21.349 IOC 1      1       
09:30:22.178 IOC 1      1       
09:30:23.478 IOC 1      1       
09:30:23.567 IOC 1      1       
09:30:26.683 IOC 1      1       
09:30:29.890 IOC 1      1       
09:30:30.155 IOC 1      1       
09:30:30.308 IOC 1      1       
09:30:30.320 IOC 1      1       
09:30:31.384 IOC 1      1       
09:30:32.263 Day 0      1       
09:30:37.249 IOC 1      2       
09:30:38.124 IOC 1      2       
09:30:38.184 IOC 1      2       
..


In [16]:
iocClusters:update duration:last time-first time by iocGroup from iocClusters where iocLoc
select from iocClusters where duration=max duration

time         tif iocLoc iocGroup duration    
---------------------------------------------
10:22:19.193 IOC 1      67       00:05:06.735
10:22:20.817 IOC 1      67       00:05:06.735
10:22:23.050 IOC 1      67       00:05:06.735
10:22:24.691 IOC 1      67       00:05:06.735
10:22:24.888 IOC 1      67       00:05:06.735
10:22:26.478 IOC 1      67       00:05:06.735
10:22:29.031 IOC 1      67       00:05:06.735
10:22:29.360 IOC 1      67       00:05:06.735
10:22:30.362 IOC 1      67       00:05:06.735
10:22:33.117 IOC 1      67       00:05:06.735
10:22:33.265 IOC 1      67       00:05:06.735
10:22:34.121 IOC 1      67       00:05:06.735
10:22:34.789 IOC 1      67       00:05:06.735
10:22:36.475 IOC 1      67       00:05:06.735
10:22:37.646 IOC 1      67       00:05:06.735
10:22:40.333 IOC 1      67       00:05:06.735
10:22:45.396 IOC 1      67       00:05:06.735
10:22:46.557 IOC 1      67       00:05:06.735
10:22:50.179 IOC 1      67       00:05:06.735
10:22:53.017 IOC 1      67       0

In [17]:
genTrades:{[n;s;st]
  dur:8*60*60*1000; / 8 hours
  / generate and sort data
  `sym`time xasc ([] time:st+n?dur; sym:n?s; tp:25+n?50f; ts:100+100*n?100; oid:(n;10)#(10*n)?.Q.an)
    }